# SageMaker Experimentsを削除

https://docs.aws.amazon.com/sagemaker/latest/dg/experiments-cleanup.html

In [ ]:
import boto3
sm = boto3.Session().client('sagemaker')

In [ ]:
def cleanup_boto3(experiment_name):
    trials = sm.list_trials(ExperimentName=experiment_name)['TrialSummaries']
    print('TrialNames:')
    for trial in trials:
        trial_name = trial['TrialName']
        print(f"\n{trial_name}")

        components_in_trial = sm.list_trial_components(TrialName=trial_name)
        print('\tTrialComponentNames:')
        for component in components_in_trial['TrialComponentSummaries']:
            component_name = component['TrialComponentName']
            print(f"\t{component_name}")
            sm.disassociate_trial_component(TrialComponentName=component_name, TrialName=trial_name)
            try:
                # comment out to keep trial components
                sm.delete_trial_component(TrialComponentName=component_name)
            except:
                # component is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        sm.delete_trial(TrialName=trial_name)
    sm.delete_experiment(ExperimentName=experiment_name)
    print(f"\nExperiment {experiment_name} deleted")

In [ ]:
for experiment_name in ['mlops-experiment01']:
    print(experiment_name)
    cleanup_boto3(experiment_name)

### experiments名を取得
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.list_experiments

In [ ]:
### experiments名一覧を出力
for i in range(len(sm.list_experiments()['ExperimentSummaries'])):
    print(sm.list_experiments()['ExperimentSummaries'][i]['ExperimentName'])